In [1]:
import libcst as cst
import re
import random
from openbugger.bugger import Bugger, bugger_example
from time import perf_counter

In [2]:
#import all bugs from openbugger
from openbugger.bugs.controlflow import ForgettingToUpdateVariableTransformer, InfiniteWhileTransformer, gen_OffByKIndexTransformer, IncorrectExceptionHandlerTransformer,MissingArgumentTransformer,ReturningEarlyTransformer
from openbugger.bugs.data import IncorrectVariableInitializationTransformer, VariableNameTypoTransformer, MutableDefaultArgumentTransformer, UseBeforeDefinitionTransformer
from openbugger.bugs.logical import gen_ComparisonTargetTransfomer, ComparisonSwapTransformer
from openbugger.bugs.type import IncorrectTypeTransformer, NonExistingMethodTransformer, SwapForTransformer
from openbugger.bugs.numpy import NumpyArrayCreationTransformer, NumpyMethodMisuseTransformer, NumpyReshapeMisuseTransformer, NumpyArangeMisuseTransformer, NumpyAxisMisuseTransformer

In [3]:
bug_dict = {"controlflow":[ForgettingToUpdateVariableTransformer, InfiniteWhileTransformer, gen_OffByKIndexTransformer(),IncorrectExceptionHandlerTransformer,MissingArgumentTransformer,ReturningEarlyTransformer],
            "data":[IncorrectVariableInitializationTransformer, VariableNameTypoTransformer, MutableDefaultArgumentTransformer, UseBeforeDefinitionTransformer],
            "logical":[gen_ComparisonTargetTransfomer(), ComparisonSwapTransformer],
            "type":[IncorrectTypeTransformer, NonExistingMethodTransformer, SwapForTransformer],
            "numpy":[NumpyArrayCreationTransformer, NumpyMethodMisuseTransformer, NumpyReshapeMisuseTransformer, NumpyArangeMisuseTransformer, NumpyAxisMisuseTransformer]}

In [4]:
import json
def load_results(file_path):
    with open(file_path, 'r') as file:
        results = json.load(file)
    return results
results = load_results('bugger_analys_results.json')
code_list = load_results('code_list.json')


In [5]:
def print_summary_statistics(results):
    for bug_type, transformers in results.items():
        print(f"\nBug Type: {bug_type}")
        for transformer, stats in transformers.items():
            print(f"\nTransformer: {transformer}")
            successful_bugs_count = len(stats["successful_bugs"])
            successful_inversions_count = len(stats["successful_inversions"])
            failed_inversions_count = len(stats["failed_inversions"])
            failed_transformations_count = len(stats["failed_transformations"])
            time_to_bug = stats["time_to_bug"]
            print(f"Successful Bugs: {successful_bugs_count}")
            print(f"Successful Inversions: {successful_inversions_count}")
            print(f"Failed Inversions: {failed_inversions_count}")
            print(f"Failed Transformations: {failed_transformations_count}")
            print(f"Time to Bug: {time_to_bug:.2f} seconds")


In [6]:
print_summary_statistics(results)



Bug Type: controlflow

Transformer: ForgettingToUpdateVariableTransformer
Successful Bugs: 2249
Successful Inversions: 2233
Failed Inversions: 16
Failed Transformations: 0
Time to Bug: 106.07 seconds

Transformer: InfiniteWhileTransformer
Successful Bugs: 654
Successful Inversions: 445
Failed Inversions: 209
Failed Transformations: 0
Time to Bug: 85.74 seconds

Transformer: OffByKIndexTransformer
Successful Bugs: 1626
Successful Inversions: 1618
Failed Inversions: 8
Failed Transformations: 0
Time to Bug: 107.97 seconds

Transformer: IncorrectExceptionHandlerTransformer
Successful Bugs: 2
Successful Inversions: 2
Failed Inversions: 0
Failed Transformations: 0
Time to Bug: 72.04 seconds

Transformer: MissingArgumentTransformer
Successful Bugs: 2161
Successful Inversions: 1899
Failed Inversions: 262
Failed Transformations: 0
Time to Bug: 104.14 seconds

Transformer: ReturningEarlyTransformer
Successful Bugs: 2325
Successful Inversions: 2319
Failed Inversions: 6
Failed Transformations: 0


In [7]:
def successful_inversions_dict(results):
    successful_inversions = {}

    for bug_type, transformers in results.items():
        for transformer, stats in transformers.items():
            for idx in stats["successful_inversions"]:
                if idx not in successful_inversions:
                    successful_inversions[idx] = set()
                successful_inversions[idx].add(transformer)

    return successful_inversions


In [8]:
successful_inversions = successful_inversions_dict(results)
for idx, transformers in successful_inversions.items():
    if idx < 10:
        print(f"Index {idx}:\n\tSuccessful Inversions: {transformers}")
    else:
        break

Index 0:
	Successful Inversions: {'UseBeforeDefinitionTransformer', 'SwapForTransformer', 'ReturningEarlyTransformer', 'ComparisonSwapTransformer', 'MissingArgumentTransformer', 'MutableDefaultArgumentTransformer', 'VariableNameTypoTransformer', 'ForgettingToUpdateVariableTransformer', 'OffByKIndexTransformer'}
Index 1:
	Successful Inversions: {'ReturningEarlyTransformer', 'IncorrectTypeTransformer', 'MissingArgumentTransformer', 'IncorrectVariableInitializationTransformer', 'VariableNameTypoTransformer', 'ForgettingToUpdateVariableTransformer'}
Index 2:
	Successful Inversions: {'ReturningEarlyTransformer', 'IncorrectTypeTransformer', 'ComparisonSwapTransformer', 'MissingArgumentTransformer', 'NonExistingMethodTransformer', 'IncorrectVariableInitializationTransformer', 'MutableDefaultArgumentTransformer', 'VariableNameTypoTransformer', 'ForgettingToUpdateVariableTransformer', 'OffByKIndexTransformer'}
Index 3:
	Successful Inversions: {'UseBeforeDefinitionTransformer', 'ReturningEarlyTr

In [9]:
from libcst.codemod import CodemodContext, Codemod
from libcst import  Module
from typing import List

import libcst as cst
from libcst.codemod import CodemodContext, ContextAwareTransformer, ContextAwareVisitor
from libcst.metadata import BatchableMetadataProvider, PositionProvider, CodePosition, CodeRange
import libcst.matchers as m
import uuid

from openbugger.context import is_modified, save_modified, PositionContextUpdater

from dataclasses import fields
from typing import Sequence

from libcst._nodes.base import CSTNode

In [20]:
class BugSelectorTransformer(ContextAwareTransformer):
        """ A transformer that inverts the changes made by other transformers that share the same context"""
        METADATA_DEPENDENCIES = (PositionProvider, )
        def __init__(self, context: CodemodContext, num_bugs_to_keep):
            super().__init__(context)
            self.num_bugs_to_keep = num_bugs_to_keep
            self.bugs_to_keep = set()

        def debug(self, tree: cst.Module) -> cst.Module:
            all_bugs = [key for key, value in self.context.scratch.items()]
            if len(all_bugs) > self.num_bugs_to_keep:
                self.bugs_to_keep = set(random.sample(all_bugs, self.num_bugs_to_keep))
            else:
                self.bugs_to_keep = set(all_bugs)
            return self.transform_module(tree)
       
        def transform_module_impl(self, tree: cst.Module) -> cst.Module:
            return tree.visit(self)

        def invert_node(self, original_node:cst.CSTNode, updated_node: cst.CSTNode) ->   cst.CSTNode:
            meta_pos = self.get_metadata(PositionProvider, original_node)
            #only updates nodes that are not already in the scratch
            already_modified  = [x for x in self.context.scratch.values() if meta_pos.start== x["original_position"].start]
            # if already_modified:
                # print("already found a node modified by",already_modified[0]["author"])
                # print("current author is",self.id)
            if already_modified and meta_pos.start not in self.bugs_to_keep:
                # print("reverting to old node",meta_pos.start, meta_pos.end)
                old_node= self.context.scratch[meta_pos.start]["original_node"]
                self.context.scratch[meta_pos.start]["debugged_node"]=old_node
                updated_node=old_node
                #pop the node from the scratch
                self.context.scratch.pop(meta_pos.start)
                # print("reverting to old node",meta_pos.start, meta_pos.end)
                # print("current node",original_node)
                # print("reverting to node",updated_node)
            return updated_node   
        def leave_ComparisonTarget(self, original_node:cst.ComparisonTarget, updated_node: cst.ComparisonTarget) -> None:
            return self.invert_node(original_node,updated_node)
        def leave_Assign(self, original_node:cst.Assign, updated_node: cst.Assign):
            return self.invert_node(original_node,updated_node)
        def leave_While(self, original_node:cst.While, updated_node: cst.While):
            return self.invert_node(original_node,updated_node) 
        def leave_Comparison(self, original_node:cst.Comparison, updated_node: cst.Comparison):
            return self.invert_node(original_node,updated_node)        
        def leave_Index(self, original_node:cst.Index, updated_node: cst.Index):
            return self.invert_node(original_node,updated_node)
        def leave_Slice(self, original_node:cst.Slice, updated_node: cst.Slice):
            return self.invert_node(original_node,updated_node)
        def leave_ExceptHandler(self, original_node:cst.ExceptHandler, updated_node: cst.ExceptHandler):
            return self.invert_node(original_node,updated_node)
        def leave_Call(self, original_node:cst.Call, updated_node: cst.Call):
            return self.invert_node(original_node,updated_node)
        def leave_Return(self, original_node:cst.Return, updated_node: cst.Return):
            return self.invert_node(original_node,updated_node)
        def leave_List(self, original_node:cst.List, updated_node: cst.List):
            return self.invert_node(original_node,updated_node)
        def leave_Dict(self, original_node:cst.Dict, updated_node: cst.Dict):
            return self.invert_node(original_node,updated_node)
        def leave_FunctionDef(self, original_node:cst.FunctionDef, updated_node: cst.FunctionDef):
            return self.invert_node(original_node,updated_node)
        def leave_Integer(self, original_node:cst.Integer, updated_node: cst.Integer):
            return self.invert_node(original_node,updated_node)
        def leave_SimpleString(self, original_node:cst.SimpleString, updated_node: cst.SimpleString):
            return self.invert_node(original_node,updated_node)
        def leave_Float(self, original_node:cst.Float, updated_node: cst.Float):
            return self.invert_node(original_node,updated_node)
        def leave_Attribute(self, original_node:cst.Attribute, updated_node: cst.Attribute):
            return self.invert_node(original_node,updated_node)
        

In [28]:
# print(random_bug[1])
# print(script)
id = 4
max_bugs = 1
script = code_list[id]
transformers = [eval(list(successful_inversions[id])[0])]
bugger = Bugger(transformers)
# Parse the script into a CST
module = cst.parse_module(script)

inverse_transformer = BugSelectorTransformer(bugger.context,max_bugs)
inverse_transformer_all = BugSelectorTransformer(bugger.context,100)
tainted = bugger.transformers[0].mutate(module)
tainted = bugger.position_updater.transform_module(tainted)
if not module.deep_equals(tainted):
    print("bug worked",bugger.context.scratch.keys())
    one_bug = inverse_transformer.debug(tainted)
    print("partial inversion completed with max bugs:",max_bugs)
    print("after_partial_inversion",bugger.context.scratch.keys())
    if not module.deep_equals(one_bug):
        print("One bug still different from original")
        clean = bugger.inverse.debug(one_bug,bugger.transformers[0].id)
        print("the debugging of one bug lead to :", module.deep_equals(clean))

    
    # 
    # tainted.code

bug worked dict_keys([CodePosition(line=3, column=15), CodePosition(line=22, column=4)])
partial inversion completed with max bugs: 1
after_partial_inversion dict_keys([CodePosition(line=22, column=4)])
One bug still different from original
the debugging of one bug lead to : True


In [15]:
transformers

['UseBeforeDefinitionTransformer']